#### Name: Rajnita Leichombam
#### USC-ID : 1370432160
#### Github-Username: Rajnita

# HOMEWORK - 3

## TIME SERIES CLASSIFICATION - PART I : Feature Creation/Extraction

(a)
Download the AReM data from:
https://archive.ics.uci.edu/ml/datasets/
Activity+Recognition+system+based+on+Multisensor+data+fusion+\%28AReM\
%29
. The dataset contains 7 folders that represent seven types of activities. In
each folder, there are multiple files each of which represents an instant of a human
performing an activity.
1
Each file containis 6 time series collected from activities
of the same person, which are called avg
rss12, var
rss12, avg
rss13, var
rss13,
vg
rss23, and ar
rss23. There are 88 instances in the dataset, each of which con-
tains 6 time series and each time series has 480 consecutive values

<span style="color:#E1721D">

The dataset is downloaded and put under '../data' folder.
    </span>

In [9]:
##ALL IMPORTS
import pandas as pd
import numpy as np
import bootstrapped.bootstrap as bs
import bootstrapped.stats_functions as bs_stats

(b)
Keep datasets 1 and 2 in folders bending1 and bending 2, as well as datasets 1,
2, and 3 in other folders as test data and other datasets as train data.

### CREATION OF TRAINING DATA AND TESTING DATA

For cycling/dataset14.csv and cycling/dataset9.csv, I removed the unnecessary ',' manually

In [10]:
list_of_folders_to_read=["bending1","bending2", "cycling","lying", "sitting", "standing","walking"]
time_series_columns=['time','avg_rss12','var_rss12','avg_rss13','var_rss13','avg_rss23','var_rss23']

##just for self check
train_file_paths=[]
training_data_files=[]
test_data_files=[]

train_dfs = list()
test_dfs = list()

def read_csv(i, directory,type_of_data):
    filename="/dataset"+str(i)+".csv"
    filepath=directory+filename
    if(type_of_data=='trainData'):
        training_data_files.append(filepath);
    else:
        ##test data
        test_data_files.append(filepath);
    if filepath=='../data/AReM/bending2/dataset4.csv':
        ##delimiting using whitespace specifically for this csv file
        df=pd.read_csv(filepath,comment='#', delim_whitespace=True, header=None);
    else:
        df=pd.read_csv(filepath,comment='#', header=None);
        
    return df;


##Used later in the notebook for 2c(i) to maintain order while printing all 88 instances, instead of merging train n test, using allinstances variable
all_instances=[]

for folder in list_of_folders_to_read:
    directory="../data/AReM/"+folder;
    if(folder.startswith('bending')):
        ##Train Data from 3,4,5...
        ##Test Data - 1,2
        ## bending1 has csv files of the format dataset<number>.csv 
        ##where number ranges from 1 to 7 in bending1 
        ##and 1 to 6 in bending2
        for i in range(1,3):
            df=read_csv(i, directory,'testData');
            df.columns=time_series_columns;
            df['activity']=folder
            test_dfs.append(df)
            all_instances.append(df)
            
        for i in range(3,7):
            df=read_csv(i, directory,'trainData');
            df.columns=time_series_columns;
            df['activity']=folder
            train_dfs.append(df)
            all_instances.append(df)

        if(folder=='bending1'):
            ##adding the 7th csv file in bending1
            df=read_csv(7, directory,'trainData');
            df.columns=time_series_columns;
            df['activity']=folder
            train_dfs.append(df)
            all_instances.append(df)
          
    else:
        ##Train Data from 4,5...
        ##Test Data - 1,2,3
        for i in range(1,4):
            df=read_csv(i, directory,'testData');
            df.columns=time_series_columns;
            df['activity']=folder
            test_dfs.append(df)
            all_instances.append(df)
        
        ##Each of the activities has 15 total sets of data
        for i in range(4,16):
            df=read_csv(i, directory,'trainData');
            df.columns=time_series_columns;
            df['activity']=folder
            train_dfs.append(df)
            all_instances.append(df)

print("\n################## TRAINING DATA #####################")
train_data = pd.concat(train_dfs, axis=0, ignore_index=True)
train_data


################## TRAINING DATA #####################


,time,avg_rss12,var_rss12,avg_rss13,var_rss13,avg_rss23,var_rss23,activity
0,0,42.00,0.71,21.25,0.43,30.00,0.00,bending1
1,250,41.50,0.50,20.25,1.48,31.25,1.09,bending1
2,500,41.50,0.50,14.25,1.92,33.00,0.00,bending1
3,750,40.75,0.83,15.75,0.43,33.00,0.00,bending1
4,1000,40.00,0.71,20.00,2.74,32.75,0.43,bending1
...,...,...,...,...,...,...,...,...
33114,118750,34.50,6.18,9.00,3.56,12.67,4.19,walking
33115,119000,25.75,6.02,13.75,2.05,16.00,1.58,walking
33116,119250,31.50,3.35,10.25,5.12,16.25,2.95,walking
33117,119500,33.75,2.77,14.00,3.24,13.75,0.43,walking


#### The above training data has 33119 rows, one of the files had only 479 rows where all others had 480 rows

In [11]:
print("\n################## TESTING DATA #####################")
test_data = pd.concat(test_dfs, axis=0, ignore_index=True)
test_data


################## TESTING DATA #####################


,time,avg_rss12,var_rss12,avg_rss13,var_rss13,avg_rss23,var_rss23,activity
0,0,39.25,0.43,22.75,0.43,33.75,1.30,bending1
1,250,39.25,0.43,23.00,0.00,33.00,0.00,bending1
2,500,39.25,0.43,23.25,0.43,33.00,0.00,bending1
3,750,39.50,0.50,23.00,0.71,33.00,0.00,bending1
4,1000,39.50,0.50,24.00,0.00,33.00,0.00,bending1
...,...,...,...,...,...,...,...,...
9115,118750,36.00,2.45,17.00,5.10,20.50,0.87,walking
9116,119000,34.33,1.89,15.00,2.45,17.00,2.12,walking
9117,119250,33.00,7.35,14.60,3.14,13.00,5.70,walking
9118,119500,31.67,1.25,11.00,6.16,19.25,2.17,walking


In [12]:
print("Total TRAINING FILES or Number of Training Instances = ",len(training_data_files))
print("\n############### LIST OF TRAINING FILES #################\n")
print(*training_data_files, sep = "\n")


Total TRAINING FILES or Number of Training Instances =  69

############### LIST OF TRAINING FILES #################

../data/AReM/bending1/dataset3.csv
../data/AReM/bending1/dataset4.csv
../data/AReM/bending1/dataset5.csv
../data/AReM/bending1/dataset6.csv
../data/AReM/bending1/dataset7.csv
../data/AReM/bending2/dataset3.csv
../data/AReM/bending2/dataset4.csv
../data/AReM/bending2/dataset5.csv
../data/AReM/bending2/dataset6.csv
../data/AReM/cycling/dataset4.csv
../data/AReM/cycling/dataset5.csv
../data/AReM/cycling/dataset6.csv
../data/AReM/cycling/dataset7.csv
../data/AReM/cycling/dataset8.csv
../data/AReM/cycling/dataset9.csv
../data/AReM/cycling/dataset10.csv
../data/AReM/cycling/dataset11.csv
../data/AReM/cycling/dataset12.csv
../data/AReM/cycling/dataset13.csv
../data/AReM/cycling/dataset14.csv
../data/AReM/cycling/dataset15.csv
../data/AReM/lying/dataset4.csv
../data/AReM/lying/dataset5.csv
../data/AReM/lying/dataset6.csv
../data/AReM/lying/dataset7.csv
../data/AReM/lying/datase

In [13]:
print("Total TEST FILES or Number of Testing Instances:",len(test_data_files))
print("\n###############LIST OF TEST FILES###############\n")
print(*test_data_files, sep = "\n")


Total TEST FILES or Number of Testing Instances: 19

###############LIST OF TEST FILES###############

../data/AReM/bending1/dataset1.csv
../data/AReM/bending1/dataset2.csv
../data/AReM/bending2/dataset1.csv
../data/AReM/bending2/dataset2.csv
../data/AReM/cycling/dataset1.csv
../data/AReM/cycling/dataset2.csv
../data/AReM/cycling/dataset3.csv
../data/AReM/lying/dataset1.csv
../data/AReM/lying/dataset2.csv
../data/AReM/lying/dataset3.csv
../data/AReM/sitting/dataset1.csv
../data/AReM/sitting/dataset2.csv
../data/AReM/sitting/dataset3.csv
../data/AReM/standing/dataset1.csv
../data/AReM/standing/dataset2.csv
../data/AReM/standing/dataset3.csv
../data/AReM/walking/dataset1.csv
../data/AReM/walking/dataset2.csv
../data/AReM/walking/dataset3.csv


In [14]:
print("TOTAL INSTANCES = ",len(train_dfs)+len(test_dfs))

TOTAL INSTANCES =  88


(c)
Feature Extraction
Classification of time series usually needs extracting features from them. In this
problem, we focus on time-domain features. <br>
i.
Research what types of time-domain features are usually used in time series
classification and list them (examples are minimum, maximum, mean, etc).

### TIME DOMAIN FEATURES USED IN TIME SERIES CLASSIFICATION

Some of the Time domain features that are used in time series classification are -<br>
1) Mean <br>
2) Standard Deviation <br>
3) Median<br>
4) Variance<br>
5) Minimum<br>
6) Maximum<br>
7)Skewnesss: measure of assymmetry of a signal<br>
8)Quartiles ( first quartile,third quartile)<br>
9) Auto Correlation -  correlation between two observations at different points in a time series<br>
10) Cross Correlation - measure of association between different time series<br>
11) Slope Sign Change - count of number of times that slope of a signal sign change<br>
12)Interquartile range<br>
13)Kurtosis: measure of peakness of a probability distribution<br>
14)Mean absolute deviation - avg of the absolute deviations of data from the data's mean<br>
15) Zero Crossing Rate - rate at which a signal changes its sign from positive to negative or vice versa within a given window<br>
16) Mean Crossing Rate - rate at which a signal changes from above mean to below mean and vice versa
<br>

<br><br>Extract the time-domain features minimum, maximum, mean, median, stan-
dard deviation, first quartile, and third quartile for all of the 6 time series
in each instance. You are free to normalize/standardize features or use them
directly.


### EXTRACTION OF TIME DOMAIN FEATURES 

In [15]:
train_dfs[0]
# all_instances=train_dfs+test_dfs
print("TOTAL INSTANCES: ",len(all_instances))

column_names_for_calculation=['avg_rss12','var_rss12','avg_rss13','var_rss13','avg_rss23','var_rss23']
time_dmn_features_all_instances=[]
columns_in_time_domain_features=[]

##Setting the column names of the dataframe with time domain features
for i in range(1,7):
    columns_in_time_domain_features.append('MIN_'+str(i));
    columns_in_time_domain_features.append('MAX_'+str(i));
    columns_in_time_domain_features.append('MEAN_'+str(i));
    columns_in_time_domain_features.append('MEDIAN_'+str(i));
    columns_in_time_domain_features.append('STD_'+str(i));
    columns_in_time_domain_features.append('FIRST_QUARTILE_'+str(i));
    columns_in_time_domain_features.append('THIRD_QUARTILE'+str(i));
                   
for instance in all_instances:
    stats_current_instance=[]
    for column in column_names_for_calculation:
        ##The order of statistics is as the order of columns created above at line 22
        stats_current_instance.append(instance[column].min())
        stats_current_instance.append(instance[column].max())
        stats_current_instance.append(instance[column].mean())
        stats_current_instance.append(instance[column].median())
        stats_current_instance.append(instance[column].std())
        stats_current_instance.append(instance[column].quantile(0.25))
        stats_current_instance.append(instance[column].quantile(0.75))
    
    time_dmn_features_all_instances.append(stats_current_instance);

time_dmn_features_dataframe=pd.DataFrame(time_dmn_features_all_instances, columns=columns_in_time_domain_features);
print("Total number of columns:",time_dmn_features_dataframe.columns.size)

print("*********************Time domain features of all instances: *********************\n")
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None
                       ):
    display(time_dmn_features_dataframe)

TOTAL INSTANCES:  88
Total number of columns: 42
*********************Time domain features of all instances: *********************



,MIN_1,MAX_1,MEAN_1,MEDIAN_1,STD_1,FIRST_QUARTILE_1,THIRD_QUARTILE1,MIN_2,MAX_2,MEAN_2,MEDIAN_2,STD_2,FIRST_QUARTILE_2,THIRD_QUARTILE2,MIN_3,MAX_3,MEAN_3,MEDIAN_3,STD_3,FIRST_QUARTILE_3,THIRD_QUARTILE3,MIN_4,MAX_4,MEAN_4,MEDIAN_4,STD_4,FIRST_QUARTILE_4,THIRD_QUARTILE4,MIN_5,MAX_5,MEAN_5,MEDIAN_5,STD_5,FIRST_QUARTILE_5,THIRD_QUARTILE5,MIN_6,MAX_6,MEAN_6,MEDIAN_6,STD_6,FIRST_QUARTILE_6,THIRD_QUARTILE6
0,37.25,45.00,40.624792,40.500,1.476967,39.2500,42.0000,0.0,1.30,0.358604,0.430,0.322605,0.0000,0.5000,4.00,29.50,19.040937,19.250,4.462952,16.0000,23.2500,0.0,7.23,0.832542,0.500,0.965659,0.0000,1.1200,27.25,38.25,34.311292,35.000,2.188449,33.0000,36.0000,0.00,1.92,0.570583,0.430,0.582915,0.0000,1.3000
1,38.00,45.67,42.812812,42.500,1.435550,42.0000,43.6700,0.0,1.22,0.372438,0.470,0.289158,0.0000,0.5000,2.00,29.50,20.096979,21.000,3.893737,19.0000,22.2500,0.0,5.76,0.876438,0.500,1.047259,0.0000,1.1450,27.67,38.50,33.024583,33.000,1.995255,32.0000,34.5000,0.00,3.11,0.571083,0.430,0.601010,0.0000,1.3000
2,35.00,47.40,43.954500,44.330,1.558835,43.0000,45.0000,0.0,1.70,0.426250,0.470,0.338690,0.0000,0.5000,6.50,29.75,22.122354,23.000,3.030943,19.7500,24.0000,0.0,4.44,0.497313,0.430,0.550657,0.0000,0.8300,29.00,38.50,35.588458,36.000,1.999604,35.3625,36.5000,0.00,1.79,0.493292,0.430,0.513506,0.0000,0.9400
3,33.00,47.75,42.179813,43.500,3.670666,39.1500,45.0000,0.0,3.00,0.696042,0.500,0.630860,0.0000,1.1200,8.50,30.00,22.183625,23.000,3.810469,20.5000,24.3725,0.0,5.15,0.989917,0.830,0.953730,0.4300,1.3000,20.00,38.67,33.493917,35.000,3.849448,30.4575,36.3300,0.00,2.18,0.613521,0.500,0.524317,0.0000,1.0000
4,33.00,45.75,41.678063,41.750,2.243490,41.3300,42.7500,0.0,2.83,0.535979,0.500,0.405469,0.4300,0.7100,3.00,28.25,19.006562,19.125,4.087107,16.5000,22.0625,0.0,6.42,0.841875,0.500,0.928801,0.4300,1.1200,23.67,37.50,29.857083,30.000,2.411026,28.4575,31.2500,0.00,1.79,0.383292,0.430,0.389164,0.0000,0.5000
5,37.00,48.00,43.454958,43.250,1.386098,42.5000,45.0000,0.0,1.58,0.378083,0.470,0.315566,0.0000,0.5000,5.75,27.00,15.793333,15.000,3.847638,13.0000,18.2700,0.0,10.03,0.849354,0.500,0.995761,0.4300,1.1200,8.00,33.50,23.034792,23.500,2.488862,22.2500,24.0000,0.00,5.26,0.679646,0.500,0.622534,0.4300,0.8700
6,36.25,48.00,43.969125,44.500,1.618364,43.3100,44.6700,0.0,1.50,0.413125,0.470,0.263111,0.4300,0.5000,1.50,26.33,15.868021,16.250,3.742420,14.2500,18.0000,0.0,5.17,0.666354,0.470,0.788985,0.0000,0.9400,11.33,30.75,22.103750,21.670,3.318301,20.5000,23.7500,0.00,2.96,0.555313,0.490,0.487826,0.0000,0.8300
7,12.75,51.00,24.562958,24.250,3.737514,23.1875,26.5000,0.0,6.87,0.590833,0.430,0.837408,0.0000,0.7100,0.00,25.33,19.121333,20.250,4.299612,18.0000,22.0000,0.0,6.76,0.736771,0.470,0.927048,0.0000,0.8700,10.67,30.00,23.493042,23.750,3.693786,20.5000,27.0000,0.00,4.97,0.700188,0.500,0.693720,0.4300,0.8700
8,0.00,42.75,27.464604,28.000,3.583582,25.5000,30.0000,0.0,7.76,0.449708,0.430,0.767197,0.0000,0.5000,7.50,35.00,20.842542,20.750,3.826268,18.0000,22.5000,0.0,5.76,0.779917,0.500,0.904436,0.0000,0.9400,0.00,33.00,17.617937,18.000,5.053642,15.0000,20.7500,0.00,6.76,1.122125,0.830,1.012342,0.4700,1.3000
9,21.00,50.00,32.586208,33.000,6.238143,26.1875,34.5000,0.0,9.90,0.516125,0.430,0.789957,0.0000,0.8225,0.00,28.25,13.941625,14.250,5.883266,10.0000,18.0000,0.0,7.40,0.868896,0.500,1.027013,0.4300,1.1200,5.00,33.75,20.345708,19.585,5.032424,17.6700,23.5000,0.00,13.61,1.162042,0.830,1.332980,0.4700,1.3000


iii.
Estimate the standard deviation of each of the time-domain features you
extracted from the data. Then, use Python’s bootstrapped or any other
method to build a 90% bootstrap confidence interval for the standard deviation
of each feature.


### BUILDING 90% BOOTSTRAP CONFIDENCE INTERVAL FOR THE STANDARD DEVIATION OF EACH FEATURE

In [16]:
confidence_intervals_all=[];
for column in columns_in_time_domain_features:
    current_feature_ci=[]
    bs_results= bs.bootstrap(np.array(time_dmn_features_dataframe[column]), alpha=0.10, stat_func=bs_stats.std)
    current_feature_ci.append(column);
    current_feature_ci.append(time_dmn_features_dataframe[column].std())
    current_feature_ci.append(bs_results.lower_bound)
    current_feature_ci.append(bs_results.upper_bound)
   
    confidence_intervals_all.append(current_feature_ci)
    
print("Number of features:",len(confidence_intervals_all))
ci_df=pd.DataFrame(confidence_intervals_all, columns=['Feature','Standard Deviation', 'Lower Bound CI','Upper Bound CI'])
ci_df


Number of features: 42


,Feature,Standard Deviation,Lower Bound CI,Upper Bound CI
0,MIN_1,9.569975,8.300660,10.810246
1,MAX_1,4.394362,3.479297,5.428480
2,MEAN_1,5.335718,4.762605,5.941910
3,MEDIAN_1,5.440054,4.849359,6.057164
4,STD_1,1.772153,1.585693,1.963669
5,FIRST_QUARTILE_1,6.153590,5.631169,6.689624
6,THIRD_QUARTILE1,5.138925,4.399915,5.912566
7,MIN_2,0.000000,0.000000,0.000000
8,MAX_2,5.062729,4.702972,5.468948
9,MEAN_2,1.574164,1.432401,1.741196


iv.
Use your judgement to select the three most important time-domain features
(one option may be min, mean, and max).

I would choose <b>Mean, Standard Deviation and Median</b> as the three most important time features.
<br>
Mean is one of the simple time domain features that considers all the values in the dataset and give an average.<br>
Standard deviation shows the measurement of variability.<br>
From the research papers that I referred (put in the REFERENCES section in the last cell), mean and standard deviation are two good features that shows the differentiation in the body postures and the intensity of the activites.<br>
At times, mean can give a skewed value and is sensitive to extreme values. <br>
For activity monitoring data set, another time feature I would consider is one of the percentile values i.e. ( min-0th percentile, max-100 percentile, median - 50 percentile, first quartile or third quartile). 
I considered  Median  as another time feature, a good measure of central tendency in case of presence of extreme values.


## ISLR 3.7.4

#### a) True Relationship is Linear, training RSS lower for linear or cubic model??

<span style="color:green">
I would expect that Cubic Regression model will have lower training RSS.
With more number of polynomial terms, we know that cubic regression model will have a closer fit. In other words, higher degrees of freedom (more polynomial terms) implies, better fit to the data. 
Here,With the same data, we are comparing linear and cubic. I believe cubic model will overfit to any non linearities that are present in the training data and hence lead to lower RSS.
    
</span>


#### b) True Relationship is Linear, testing RSS lower for linear or cubic model??

<span style="color:green">
Given that the true relationship between X and Y is linear, in case of test data, I would expect the linear regressor to perform better i.e. lesser test RSS. Since the true model is linear, the cubic model may have overfitted the training data, and with test data the cubic model will likely have more error i.e. more test RSS.
</span>

#### c) True Relationship is not Linear (no idea how far), training RSS lower for linear or cubic model??

<span style="color:green">
A polynomial regression model will lead to closer fit as a result of greater flexibility compared to linear fit. Similar to (a), CUBIC fit will have LESSER TRAINING RSS, regardless of the information that true relationship is linear or non linear , as we are dealing with training data here.

</span>

#### d) True Relationship is not Linear (no idea how far), test RSS lower for linear or cubic model??


<span style="color:green">
I would say, it depends on how non linear the true relationship is. <br>
If the relationship is non linear but closer to linear, then I would expect linear regression to have a LOWER TEST RSS. In this case, cubic regression model would overfit and hence higher RSS.
<br>
If the relationship is more of a cubic nature than linear, then cubic regression model will have lesser RSS. Linear Regression model can underfit in this case and hence,higher RSS value.

</span>



REFERENCES:

https://pypi.org/project/bootstrapped/
https://revistia.org/files/articles/ejis_v2_i3s_16/Cemil.pdf
https://papers.ssrn.com/sol3/papers.cfm?abstract_id=4014024 https://www.researchgate.net/publication/298911566_Human_activity_recognition_using_multisensor_data_fusion_based_on_Reservoir_Computing 
https://www.sciencedirect.com/science/article/pii/S0031320310001950
https://ieeexplore-ieee-org.libproxy2.usc.edu/stamp/stamp.jsp?tp=&arnumber=7250435    https://www.researchgate.net/publication/258029665_Multisensor_Data_Fusion_for_Activity_Recognition_Based_on_Reservoir_Computing
 https://www.researchgate.net/figure/Time-domain-features_tbl1_306269759
                        